### Import Required Libraries and Set Up Environment Variables

In [5]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('example.env')

### CME Data

In [3]:
base_url = "https://api.nasa.gov/DONKI/"
CME = "CME"

# Define the date range for CME data search
startDate = "2013-05-01"
endDate = "2024-05-01"

# Construct the full URL with parameters
url = f"{base_url}{CME}?api_key={NASA_API_KEY}&startDate={startDate}endDate={endDate}"
print("Constructed URL:", url)



In [4]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(url)

# Check if the request was successful
if cme_response.status_code == 200:
    # Parse the JSON response
    cme_data = cme_response.json()
    print("CME data fetched successfully.")
else:
    print(f"Failed to fetch data: {cme_response.status_code}")



In [4]:
# Convert the response variable to json and store it as a variable named cme_json
if cme_response.status_code == 200:
    cme_json = cme_response.json()
else:
    cme_json = None
    print("Failed to convert response to JSON.")




In [5]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data

if cme_json and isinstance(cme_json, list) and len(cme_json) > 0:
    first_result = cme_json[0]  # Get the first result from the list
    formatted_result = json.dumps(first_result, indent=4)  # Format with indent
    print("First CME Result:")
    print(formatted_result)
else:
    print("No CME data available to preview.")


[
    {
        "activityID": "2013-05-01T03:12:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2013-05-01T03:12Z",
        "sourceLocation": "",
        "activeRegionNum": null,
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
        "note": "",
        "submissionTime": "2013-08-07T16:54Z",
        "instruments": [
            {
                "displayName": "SOHO: LASCO/C2"
            },
            {
                "displayName": "SOHO: LASCO/C3"
            },
            {
                "displayName": "STEREO A: SECCHI/COR2"
            },
            {
                "displayName": "STEREO B: SECCHI/COR2"
            }
        ],
        "cmeAnalyses": [
            {
                "isMostAccurate": true,
                "time21_5": "2013-05-01T07:07Z",
                "latitude": 12.0,
                "longitude": -120.0,
                "halfAngle": 36.0,
                "speed": 860.0,
                "type": "C",
   

In [6]:
# Convert cme_json to a Pandas DataFrame 

# Keep only the columns: activityID, startTime, linkedEvents

if cme_json:
    df_cme = pd.DataFrame(cme_json)  # Create a DataFrame from the JSON data
    df_cme_filtered = df_cme[['activityID', 'startTime', 'linkedEvents']]  # Keep only the specified columns
    print("CME DataFrame created with filtered columns:")
    print(df_cme_filtered.head())  # Display the first few rows of the filtered DataFrame
else:
    print("No CME data available to convert to DataFrame.")



activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-02T05:24:00-CME-001  2013-05-02T05:24Z   
2  2013-05-02T14:36:00-CME-001  2013-05-02T14:36Z   
3  2013-05-03T18:00:00-CME-001  2013-05-03T18:00Z   
4  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   

                                      linkedEvents  
0  [{'activityID': '2013-05-04T04:52:00-IPS-001'}]  
1                                             None  
2                                             None  
3                                             None  
4  [{'activityID': '2013-05-07T04:37:00-IPS-001'}]

In [7]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
df_cme_filtered = df_cme_filtered.dropna(subset=['linkedEvents'])

# print("Rows with missing 'linkedEvents' removed.")
print(df_cme_filtered)  # Display the number of remaining rows




activityID          startTime  \
0   2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
4   2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   
7   2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z   
10  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   
13  2013-05-13T16:18:00-CME-001  2013-05-13T16:18Z   

                                         linkedEvents  
0     [{'activityID': '2013-05-04T04:52:00-IPS-001'}]  
4     [{'activityID': '2013-05-07T04:37:00-IPS-001'}]  
7     [{'activityID': '2013-05-12T23:30:00-IPS-001'}]  
10  [{'activityID': '2013-05-13T01:53:00-FLR-001'}...  
13  [{'activityID': '2013-05-13T15:40:00-FLR-001'}...

In [8]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows


# Iterate over each index in the DataFrame

    # Iterate over each dictionary in the list
    
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
      
# Create a new DataFrame from the expanded rows

expanded_rows = []  # Initialize an empty list to store the expanded rows

# Iterate over each index in the DataFrame
for index in df_cme_filtered.index:
    activity_id = df_cme_filtered.at[index, 'activityID']  # Get the 'activityID' for the current row
    start_time = df_cme_filtered.at[index, 'startTime']  # Get the 'startTime' for the current row
    linked_events = df_cme_filtered.at[index, 'linkedEvents']  # Get the 'linkedEvents' for the current row

    # If linked_events is not empty, iterate over each event
    if linked_events:
        for event in linked_events:
            # Append a new dictionary for each event with corresponding 'activityID' and 'startTime'
            expanded_rows.append({
                'activityID': activity_id,
                'startTime': start_time,
                'linkedEvent': event  # Add the individual linked event
            })

# Create a new DataFrame from the expanded rows
df_expanded = pd.DataFrame(expanded_rows)

print("Expanded DataFrame created with each linked event as a separate row:")
print(df_expanded.head())  # Display the first few rows of the expanded DataFrame


activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   
2  2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z   
3  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   
4  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   

                                    linkedEvents  
0  {'activityID': '2013-05-04T04:52:00-IPS-001'}  
1  {'activityID': '2013-05-07T04:37:00-IPS-001'}  
2  {'activityID': '2013-05-12T23:30:00-IPS-001'}  
3  {'activityID': '2013-05-13T01:53:00-FLR-001'}  
4  {'activityID': '2013-05-13T04:12:00-SEP-001'}

In [9]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors

        # Log the error or print it for debugging


def extract_activityID_from_dict(linked_event_dict):
    try:
        # Assuming linked_event_dict contains a key for activityID
        activity_id = linked_event_dict.get('activityID', None)  # Get activityID, default to None if not found
        if activity_id is None:
            raise ValueError("activityID not found in the provided dictionary.")
        return activity_id
    except Exception as e:
        # Log the error for debugging
        print(f"Error extracting activityID: {e}")
        return None  # Return None if there's an error

# Example usage to test the function
example_linked_event = {
    'activityID': 'CME123',
    'eventType': 'CME',
    'startTime': '2023-01-01T00:00:00Z'
}

# Test the function
extracted_activity_id = extract_activityID_from_dict(example_linked_event)
print("Extracted activityID:", extracted_activity_id)  # Should output: Extracted activityID: CME123


'2013-05-04T04:52:00-IPS-001'

In [10]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
df_expanded['GST_ActivityID'] = df_expanded['linkedEvent'].apply(
    lambda x: extract_activityID_from_dict(x) if isinstance(x, dict) else None
)

# Preview the updated DataFrame
print(df_expanded[['activityID', 'startTime', 'linkedEvent', 'GST_ActivityID']].head())

activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   
2  2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z   
3  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   
4  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   

                                    linkedEvents               GST_ActivityID  
0  {'activityID': '2013-05-04T04:52:00-IPS-001'}  2013-05-04T04:52:00-IPS-001  
1  {'activityID': '2013-05-07T04:37:00-IPS-001'}  2013-05-07T04:37:00-IPS-001  
2  {'activityID': '2013-05-12T23:30:00-IPS-001'}  2013-05-12T23:30:00-IPS-001  
3  {'activityID': '2013-05-13T01:53:00-FLR-001'}  2013-05-13T01:53:00-FLR-001  
4  {'activityID': '2013-05-13T04:12:00-SEP-001'}  2013-05-13T04:12:00-SEP-001

In [11]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:

df_expanded = df_expanded.dropna(subset=['GST_ActivityID'])

# Preview the updated DataFrame after removing rows with missing GST_ActivityID
print(df_expanded.head())  # Display the first few rows of the updated DataFrame




In [12]:
# print out the datatype of each column in this DataFrame:
print("Data types of each column in the DataFrame:")
print(df_expanded.dtypes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1807 entries, 0 to 1806
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   activityID      1807 non-null   object
 1   startTime       1807 non-null   object
 2   linkedEvents    1807 non-null   object
 3   GST_ActivityID  1807 non-null   object
dtypes: object(4)
memory usage: 56.6+ KB


In [13]:
# Convert the 'GST_ActivityID' column to string format 

# Convert startTime to datetime format  

# Rename startTime to startTime_CME and activityID to cmeID

# Drop linkedEvents

# Verify that all steps were executed correctly
df_expanded['GST_ActivityID'] = df_expanded['GST_ActivityID'].astype(str)

# Convert startTime to datetime format
df_expanded['startTime'] = pd.to_datetime(df_expanded['startTime'])

# Rename columns
df_expanded.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'}, inplace=True)

# Drop the 'linkedEvents' column
df_expanded.drop(columns=['linkedEvent'], inplace=True)

# Verify that all steps were executed correctly
print("DataFrame after transformations:")
print(df_expanded.head())  # Display the first few rows to verify changes
print("\nData types of each column in the DataFrame after transformations:")
print(df_expanded.dtypes)  # Check the data types to ensure the transformations


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1807 entries, 0 to 1806
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           1807 non-null   object             
 1   startTime_CME   1807 non-null   datetime64[ns, UTC]
 2   GST_ActivityID  1807 non-null   string             
dtypes: datetime64[ns, UTC](1), object(1), string(1)
memory usage: 42.5+ KB


In [14]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
filtered_df = df_expanded[df_expanded['GST_ActivityID'].str.contains('GST', na=False)]

# Verify the result
print("Filtered DataFrame containing only CMEs related to GSTs:")
print(filtered_df.head())  # Display the first few rows of the filtered DataFrame
print(f"\nNumber of rows after filtering: {len(filtered_df)}")  # Display the number of rows after filtering



cmeID             startTime_CME  \
21   2013-06-02T20:24:00-CME-001 2013-06-02 20:24:00+00:00   
48   2013-09-29T22:40:00-CME-001 2013-09-29 22:40:00+00:00   
90   2013-12-04T23:12:00-CME-001 2013-12-04 23:12:00+00:00   
148  2014-02-16T14:15:00-CME-001 2014-02-16 14:15:00+00:00   
151  2014-02-18T01:25:00-CME-001 2014-02-18 01:25:00+00:00   

                  GST_ActivityID  
21   2013-06-07T03:00:00-GST-001  
48   2013-10-02T03:00:00-GST-001  
90   2013-12-08T00:00:00-GST-001  
148  2014-02-19T03:00:00-GST-001  
151  2014-02-20T03:00:00-GST-001

### GST Data

In [15]:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST)
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate = "2024-05-01"

# Build URL for GST
url_gst = f"{base_url}{GST}?api_key={NASA_API_KEY}&startDate={startDate}endDate={endDate}"

# Print the constructed URL
print("Constructed URL for GST:", url_gst)


In [16]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(url_gst)

# Check if the request was successful
if gst_response.status_code == 200:
    print("GST data fetched successfully.")
else:
    print(f"Failed to fetch GST data: {gst_response.status_code}")


In [17]:
# Convert the response variable to json and store it as a variable named gst_json

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
if gst_response.status_code == 200:
    gst_json = gst_response.json()  # Convert the response to JSON
else:
    gst_json = None
    print("Failed to convert response to JSON.")

# Preview the first result in JSON format
if gst_json and isinstance(gst_json, list) and len(gst_json) > 0:
    first_result_gst = gst_json[0]  # Get the first result from the list
    formatted_result_gst = json.dumps(first_result_gst, indent=4)  # Format with indent
    print("First GST Result:")
    print(formatted_result_gst)
else:
    print("No GST data available to preview.")



[
    {
        "gstID": "2013-06-01T01:00:00-GST-001",
        "startTime": "2013-06-01T01:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-01T01:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "linkedEvents": [
            {
                "activityID": "2013-05-31T15:45:00-HSS-001"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
        "submissionTime": "2013-07-15T19:26Z"
    }
]


In [18]:
# Convert gst_json to a Pandas DataFrame  

# Keep only the columns: activityID, startTime, linkedEvents
if gst_json:
    # Create a DataFrame from the JSON data
    df_gst = pd.DataFrame(gst_json)

    # Keep only the columns that exist: gstID, startTime, linkedEvents
    columns_to_keep = ['gstID', 'startTime', 'linkedEvents']
    df_gst_filtered = df_gst[columns_to_keep].dropna(subset=['linkedEvents'])  # Drop rows with missing linkedEvents

    print("GST DataFrame created with filtered columns:")
    print(df_gst_filtered.head())  # Display the first few rows of the filtered DataFrame
else:
    print("No GST data available to convert to DataFrame.")



gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2  2013-06-29T03:00:00-GST-001  2013-06-29T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-12-08T00:00:00-GST-001  2013-12-08T00:00Z   

                                        linkedEvents  
0    [{'activityID': '2013-05-31T15:45:00-HSS-001'}]  
1    [{'activityID': '2013-06-02T20:24:00-CME-001'}]  
2                                               None  
3  [{'activityID': '2013-09-29T22:40:00-CME-001'}...  
4  [{'activityID': '2013-12-04T23:12:00-CME-001'}...

In [19]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
df_gst_filtered = df_gst_filtered.dropna(subset=['linkedEvents'])

# Print the number of remaining rows after removal
print(df_gst_filtered)



gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-12-08T00:00:00-GST-001  2013-12-08T00:00Z   
5  2014-02-19T03:00:00-GST-001  2014-02-19T03:00Z   

                                        linkedEvents  
0    [{'activityID': '2013-05-31T15:45:00-HSS-001'}]  
1    [{'activityID': '2013-06-02T20:24:00-CME-001'}]  
3  [{'activityID': '2013-09-29T22:40:00-CME-001'}...  
4  [{'activityID': '2013-12-04T23:12:00-CME-001'}...  
5  [{'activityID': '2014-02-16T14:15:00-CME-001'}...

In [20]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
if 'linkedEvents' in df_gst_filtered.columns:
    df_gst_exploded = df_gst_filtered.explode('linkedEvents').reset_index(drop=True)  # Explode and reset the index
    df_gst_exploded = df_gst_exploded.dropna(subset=['linkedEvents'])  # Drop rows with missing linkedEvents

    print(df_gst_exploded.head())  # Display the first few rows of the exploded DataFrame
else:
    print("linkedEvents column not found in the DataFrame.")


In [21]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
df_gst_exploded['CME_ActivityID'] = df_gst_exploded['linkedEvents'].apply(
    lambda x: extract_activityID_from_dict(x) if isinstance(x, dict) else None
)

# Remove rows with missing CME_ActivityID
df_gst_exploded = df_gst_exploded.dropna(subset=['CME_ActivityID'])
# Preview the updated DataFrame
print("DataFrame after adding CME_ActivityID and removing missing values:")
print(df_gst_exploded.head())



gstID          startTime  \
0  2013-06-01T01:00:00-GST-001  2013-06-01T01:00Z   
1  2013-06-07T03:00:00-GST-001  2013-06-07T03:00Z   
2  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
3  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   
4  2013-10-02T03:00:00-GST-001  2013-10-02T03:00Z   

                                    linkedEvents               CME_ActivityID  
0  {'activityID': '2013-05-31T15:45:00-HSS-001'}  2013-05-31T15:45:00-HSS-001  
1  {'activityID': '2013-06-02T20:24:00-CME-001'}  2013-06-02T20:24:00-CME-001  
2  {'activityID': '2013-09-29T22:40:00-CME-001'}  2013-09-29T22:40:00-CME-001  
3  {'activityID': '2013-10-02T01:54:00-IPS-001'}  2013-10-02T01:54:00-IPS-001  
4  {'activityID': '2013-10-02T02:47:00-MPC-001'}  2013-10-02T02:47:00-MPC-001

In [22]:
# Convert the 'CME_ActivityID' column to string format 

# Convert the 'gstID' column to string format 

# Convert startTime to datetime format  

# Rename startTime to startTime_GST 

# Drop linkedEvents

# Verify that all steps were executed correctly
df_gst_exploded['CME_ActivityID'] = df_gst_exploded['CME_ActivityID'].astype(str)

# Convert the 'gstID' column to string format
df_gst_exploded['gstID'] = df_gst_exploded['gstID'].astype(str)

# Convert startTime to datetime format
df_gst_exploded['startTime'] = pd.to_datetime(df_gst_exploded['startTime'])

# Rename startTime to startTime_GST
df_gst_exploded.rename(columns={'startTime': 'startTime_GST'}, inplace=True)

# Drop linkedEvents
df_gst_exploded.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
print("Data types after modifications:")
print(df_gst_exploded.dtypes)

print("\nFirst few rows of the modified DataFrame:")
print(df_gst_exploded.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           217 non-null    string             
 1   startTime_GST   217 non-null    datetime64[ns, UTC]
 2   CME_ActivityID  217 non-null    string             
dtypes: datetime64[ns, UTC](1), string(2)
memory usage: 5.2 KB


In [23]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
df_gst_cme_filtered = df_gst_exploded[df_gst_exploded['CME_ActivityID'].str.contains('CME', na=False)]

# Verify the results
print(f"Filtered DataFrame shape: {df_gst_cme_filtered.shape}")
print("First few rows of the filtered DataFrame:")
print(df_gst_cme_filtered.head())


# Before merging, rename startTime in df_cme
df_cme.rename(columns={'startTime': 'startTime_CME'}, inplace=True)


gstID             startTime_GST  \
1  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
2  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
5  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
7  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
9  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   

                CME_ActivityID  
1  2013-06-02T20:24:00-CME-001  
2  2013-09-29T22:40:00-CME-001  
5  2013-12-04T23:12:00-CME-001  
7  2014-02-16T14:15:00-CME-001  
9  2014-02-18T01:25:00-CME-001

### Merge both datatsets



In [24]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(
    df_expanded,
    df_gst_exploded,
    left_on='GST_ActivityID',
    right_on='CME_ActivityID',
    how='inner'
)

# Preview the merged DataFrame
print("Merged DataFrame:")
print(merged_df.head())

gstID             startTime_GST  \
0  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
1  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
2  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
3  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
4  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   

                CME_ActivityID                        cmeID  \
0  2013-06-02T20:24:00-CME-001  2013-06-02T20:24:00-CME-001   
1  2013-09-29T22:40:00-CME-001  2013-09-29T22:40:00-CME-001   
2  2013-12-04T23:12:00-CME-001  2013-12-04T23:12:00-CME-001   
3  2014-02-16T14:15:00-CME-001  2014-02-16T14:15:00-CME-001   
4  2014-02-18T01:25:00-CME-001  2014-02-18T01:25:00-CME-001   

              startTime_CME               GST_ActivityID  
0 2013-06-02 20:24:00+00:00  2013-06-07T03:00:00-GST-001  
1 2013-09-29 22:40:00+00:00  2013-10-02T03:00:00-GST-001  
2 2013-12-04 23:12:00+00:00  2013-12-08T00:00:00-GST-001  
3 2014-02-16 14:15:00+00:00  2014-02-19T03:00:00-GST-001  
4 2014-02-18 01:25:00+00:00  2014-02-20T03:00:00-GST-001

In [25]:
# Verify that the new DataFrame has the same number of rows as cme and gst
merged_df['timeDiff'] = merged_df['startTime_GST'] - merged_df['startTime_CME']

# Preview the updated DataFrame with the new timeDiff column
print("DataFrame with time difference:")
print(merged_df[['GST_ActivityID', 'CME_ActivityID', 'startTime_CME', 'startTime_GST', 'timeDiff']].head())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           67 non-null     string             
 1   startTime_GST   67 non-null     datetime64[ns, UTC]
 2   CME_ActivityID  67 non-null     object             
 3   cmeID           67 non-null     object             
 4   startTime_CME   67 non-null     datetime64[ns, UTC]
 5   GST_ActivityID  67 non-null     string             
dtypes: datetime64[ns, UTC](2), object(2), string(2)
memory usage: 3.3+ KB


### Computing the time it takes for a CME to cause a GST

In [26]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
import pandas as pd

# Sample data
data = {
    'startTime_GST': ['2024-10-28 12:00:00', '2024-10-28 14:00:00'],
    'startTime_CME': ['2024-10-28 11:00:00', '2024-10-28 15:30:00']
}

# Create DataFrame
df = pd.DataFrame(data)

# Convert columns to datetime
df['startTime_GST'] = pd.to_datetime(df['startTime_GST'])
df['startTime_CME'] = pd.to_datetime(df['startTime_CME'])

# Compute the time difference
df['timeDiff'] = df['startTime_GST'] - df['startTime_CME']

# Display the updated DataFrame
print(df)



gstID             startTime_GST  \
0  2013-06-07T03:00:00-GST-001 2013-06-07 03:00:00+00:00   
1  2013-10-02T03:00:00-GST-001 2013-10-02 03:00:00+00:00   
2  2013-12-08T00:00:00-GST-001 2013-12-08 00:00:00+00:00   
3  2014-02-19T03:00:00-GST-001 2014-02-19 03:00:00+00:00   
4  2014-02-20T03:00:00-GST-001 2014-02-20 03:00:00+00:00   

                CME_ActivityID                        cmeID  \
0  2013-06-02T20:24:00-CME-001  2013-06-02T20:24:00-CME-001   
1  2013-09-29T22:40:00-CME-001  2013-09-29T22:40:00-CME-001   
2  2013-12-04T23:12:00-CME-001  2013-12-04T23:12:00-CME-001   
3  2014-02-16T14:15:00-CME-001  2014-02-16T14:15:00-CME-001   
4  2014-02-18T01:25:00-CME-001  2014-02-18T01:25:00-CME-001   

              startTime_CME               GST_ActivityID        timeDiff  
0 2013-06-02 20:24:00+00:00  2013-06-07T03:00:00-GST-001 4 days 06:36:00  
1 2013-09-29 22:40:00+00:00  2013-10-02T03:00:00-GST-001 2 days 04:20:00  
2 2013-12-04 23:12:00+00:00  2013-12-08T00:00:00-GST-001 3 days 00:48:00  
3 2014-02-16 14:15:00+00:00  2014-02-19T03:00:00-GST-001 2 days 12:45:00  
4 2014-02-18 01:25:00+00:00  2014-02-20T03:00:00-GST-001 2 days 01:35:00

In [27]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
# Use describe() to compute the mean and median time
time_diff_summary = df['timeDiff'].describe()

# Extract mean and median
mean_time = time_diff_summary['mean']
median_time = time_diff_summary['50%']  # 50% is the median

# Display results
print(f"Mean time for CME to cause GST: {mean_time}")
print(f"Median time for CME to cause GST: {median_time}")


timeDiff
count                         67
mean   2 days 21:35:13.432835820
std    1 days 00:02:46.681279427
min              1 days 05:36:00
25%              2 days 03:12:00
50%              2 days 17:48:00
75%              3 days 12:17:00
max              6 days 03:00:00

### Exporting data in csv format

In [28]:
# Export data to CSV without the index
merged_df.to_csv('merged_data.csv', index=False)

print("Data exported to 'merged_data.csv' without index.")
